In [5]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.11.1 --progress-bar off


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects
ERROR: Could not find a version that satisfies the requirement auto-gptq==0.4.2 (from versions: 0.0.4, 0.0.5, 0.1.0, 0.2.0, 0.2.1, 0.2.2, 0.3.0, 0.3.1, 0.3.2, 0.5.0, 0.5.1, 0.6.0)
ERROR: No matching distribution found for auto-gptq==0.4.2


In [7]:
import pandas as pd
import json


In [8]:
from huggingface_hub import login
login()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/harshul/.cache/huggingface/token
Login successful


In [9]:
# import torch
# from langchain import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
# #MODEL_NAME = "meta-llama/Llama-2-7b-chat"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
# )

# generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
# generation_config.max_new_tokens = 1024
# generation_config.temperature = 0.0001
# generation_config.top_p = 0.95
# generation_config.do_sample = True
# generation_config.repetition_penalty = 1.15

# text_pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     return_full_text=True,
#     generation_config=generation_config,
# )

# llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.001})

In [10]:
from langchain.llms import Ollama

llm = Ollama(model="mistral:instruct")

In [15]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or 
"no", for every symptom. The symptoms are as follows:
[Little interest or pleasure in doing things,

Feeling down, depressed, or hopeless, 

Trouble falling or staying asleep, or sleeping too much, 
Feeling tired or having little energy, 

Poor appetite or overeating, 

Feeling bad about yourself or that you are a failure or
have let yourself or your family down, 

Trouble concentrating on things, such as reading the
newspaper or watching television,

Moving or speaking so slowly that other people could
have noticed. Or the opposite being so figety or
restless that you have been moving around a lot more
than usual,

Thoughts that you would be better off dead, or of
hurting yourself] .

Give the output as a python list of lists, with the list containing the symptom and the
answer, separated by a comma.
{text}:
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [16]:
text = """
Even during my depression I’ll get these small mini bursts of optimism and motivation thinking, “maybe I can make it?! Maybe I can improve my life and be happy!” And I’ll go to the gym for one day, or run, or try to learn something new, but that’s short lasted. The second I complete the task all I can think is. “What’s the point? I’m probably going to die anyway, so why spend time improving myself?”

I honestly have no will to even live life, all my energy is spent just existing by distracting myself and spending money on stupid shit just to forget about life.

I feel stuck in purgatory. Not courageous enough to kill myself, not motivated enough to make myself happy. I’m stuck. Existing.
"""

In [18]:
print(prompt.format(text=text))


<s>[INST] <<SYS>>
For a given user post sentence, does it show signs of the symptom. Answer in binary "yes" or 
"no", for every symptom. The symptoms are as follows:
[Little interest or pleasure in doing things,

Feeling down, depressed, or hopeless, 

Trouble falling or staying asleep, or sleeping too much, 
Feeling tired or having little energy, 

Poor appetite or overeating, 

Feeling bad about yourself or that you are a failure or
have let yourself or your family down, 

Trouble concentrating on things, such as reading the
newspaper or watching television,

Moving or speaking so slowly that other people could
have noticed. Or the opposite being so figety or
restless that you have been moving around a lot more
than usual,

Thoughts that you would be better off dead, or of
hurting yourself] .

Give the output as a python list of lists, with the list containing the symptom and the
answer, separated by a comma.

Even during my depression I’ll get these small mini bursts of optimism an

In [19]:
%%time
result = llm(prompt.format(text=text))

CPU times: user 96.8 ms, sys: 558 ms, total: 655 ms
Wall time: 1min 10s


In [20]:
print(result)

 [["Little interest or pleasure in doing things", "yes"],
 ["Feeling down, depressed, or hopeless", "yes"],
 ["Trouble falling or staying asleep, or sleeping too much", "no"],
 ["Feeling tired or having little energy", "yes"],
 ["Poor appetite or overeating", "yes"],
 ["Feeling bad about yourself or that you are a failure or have let yourself or your family down", "yes"],
 ["Trouble concentrating on things, such as reading the newspaper or watching television", "yes"],
 ["Moving or speaking so slowly that other people could have noticed. Or the opposite being so figety or restless that you have been moving around a lot more than usual", "no"],
 ["Thoughts that you would be better off dead, or of hurting yourself", "yes"]]


In [ ]:
# with open('/kaggle/input/primate/primate_dataset.json', 'r') as file:
#     data = json.load(file)

# Initialize lists for columns
post_titles = []
second_columns = []
i=0
# Process each entry in the JSON data

import pandas as pd
df = pd.read_csv('prompt_dataset_primate.csv')

for index, row in df.iterrows():
    # Extract post_title and post_text
    print(index)
    post_title = row['Title']
    post_text = row['Text']
    print(f'TITLE: {post_title}')
    #print(f'TEXT: {post_text}')

    # Process post_text using the get_ans function
    second_column_value = llm(prompt.format(text=post_text))

    # Append values to lists
    post_titles.append(post_title)
    second_columns.append(second_column_value)

# Create a DataFrame
df = pd.DataFrame({'Title': post_titles, 'Output': second_columns})

# Write DataFrame to CSV
#df.to_csv('abc.csv', index=False)


0
TITLE: How am I supposed to improve my life when I constantly want to die
1
TITLE: Should I click yes to give the psychiatrist permission to share my information with one of my parents? What are they gonna share?
2
TITLE: What would you do in my position?
3
TITLE: SO says it's depression linked. How to stop being too much?
4
TITLE: Your experience with SSRI's
5
TITLE: Two Faced People - Need Advice
6
TITLE: How can I force myself to do important things when I’m feeling like shit?
7
TITLE: I feel like i'm loosing my best friend and I don't know what to do
8
TITLE: I hate myself so much, I don't deserve to live
9
TITLE: My reward system is fked
10
TITLE: My dad just threatened me for crying and going to my room and i would really need some kind words
11
TITLE: Teenager with no hope.


In [ ]:
# df.to_csv('mistral_zeroshot_rerun.csv', index=False)
# print('SUCCESS')